## This notebook is just to test the various functions of the project

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from unstructured.partition.image import partition_image
from PIL import Image
import io
from langchain_groq import ChatGroq

In [ ]:
# Initialize the ChatGroq model
chat = ChatGroq(
    model="llama-3.2-11b-vision-preview", # you can use an of the suited models
    temperature=0.6,
    max_tokens=1024,
    timeout=None,
    max_retries=2,
    groq_api_key='groq_api_key'  # Replace with your Groq API key
)

# Function to scrape the page content from the item link
def scrape(item_link):
    """Scrapes the item page content from the given link and cleans it up."""
    loader = WebBaseLoader(item_link)
    page_data = loader.load()

    if not page_data:
        return "No content found or the page could not be loaded."

    # Get the page content
    page_content = page_data.pop().page_content

    # Remove unnecessary newlines and extra spaces
    cleaned_content = ' '.join(page_content.split())

    return cleaned_content

# Function to merge multiple images into one image
def merge_images(image_paths, direction='vertical'):
    """Merge multiple images into one image."""
    images = [Image.open(image_path) for image_path in image_paths]

    if direction == 'vertical':
        total_height = sum(img.height for img in images)
        max_width = max(img.width for img in images)
        merged_image = Image.new('RGB', (max_width, total_height))
        y_offset = 0
        for img in images:
            merged_image.paste(img, (0, y_offset))
            y_offset += img.height
    elif direction == 'horizontal':
        total_width = sum(img.width for img in images)
        max_height = max(img.height for img in images)
        merged_image = Image.new('RGB', (total_width, max_height))
        x_offset = 0
        for img in images:
            merged_image.paste(img, (x_offset, 0))
            x_offset += img.width

    return merged_image

# Function to extract text from images
def image_to_text(image):
    """Extracts text from the given image."""
    img_byte_arr = io.BytesIO()
    image.save(img_byte_arr, format='PNG')  # Save image to a byte array
    img_byte_arr.seek(0)  # Reset buffer position
    elements = partition_image(file=img_byte_arr, languages=["eng", "kan"], strategy="auto")
    return elements


In [ ]:

# Main function to execute the workflow
if __name__ == "__main__":
    # Example URL to scrape
    item_link = "https://www.amazon.in/Macaroni-Delicious-Italian-Instant-Ingredients/dp/B006JIREEW"

    # Step 1: Scrape product specifications from the website
    scraped_data = scrape(item_link)

    # Step 2: Check for access restrictions
    if len(scraped_data) > 50 and "access denied" not in scraped_data.lower():
        # Step 3: Paths to images for OCR
        img_paths = [
            "/content/drive/My Drive/SAMPLE-DATA/fig-1.png",
            "/content/drive/My Drive/SAMPLE-DATA/fig-2.png",
        ]

        # Step 4: Merge images
        merged_image = merge_images(img_paths, direction='vertical')  # Merge images

        # Step 5: Perform OCR on the merged image
        img_txt = image_to_text(merged_image)

        # Step 6: Convert img_txt elements to a single string
        img_txt_str = " ".join([str(element.text) for element in img_txt if hasattr(element, 'text')])

        # Step 7: Prepare the prompt for comparison
        prompt = (
            f"You are an AI assistant that helps to find the differences between the real product specifications "
            f"from the website and the user given image data. \n\n"
            f"Site Data:\n{scraped_data}\n\n"
            f"Image Data:\n{img_txt_str}\n\n"
            "Please analyze the specifications mentioned in the site data as part of the image data and give your analysis report in detail"
        )

        # Step 8: Define the message payload for sending
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                ],
            }
        ]

        # Step 9: Send the message to the model
        response = chat.invoke(messages)
        print(response.content)
    else:
        print("Website has restricted access or insufficient content.")
